# **A. - B. ADELANTE Y ATRAS**

In [ ]:
#Juan Sebastian Camargo Sanchez
#Juan Pablo Daza Pereira

import numpy as np
import sys
#------------------------------
def initialize_parameters(layers):
    red = {}
    for i in range(0,len(layers)):
      if i>0:
        auxW=np.random.randn(layers[i],layers[i-1])*0.01
        auxB=np.zeros((layers[i],1))
        red["W"+str(i)]=auxW
        red["b"+str(i)]=auxB
    return red
#------------------------------
def forward_step(A_1, W, b):
  zeta=W@A_1 + b
  zeta = zeta.astype(np.float128)
  A=1/(1+np.exp(-zeta))
  return A,np.array([A_1,W,A],dtype=object)
#------------------------------
def forward(X, parameters):
  A_1=X
  caches=[]
  for i in range(1,int(len(parameters)/2)+1):
    A,cache=forward_step(A_1,parameters["W"+str(i)],parameters["b"+str(i)])
    A_1 = cache[2]
    caches.append(cache)
  return A,caches
#------------------------------
def predict(X, parameters):
  L = len(parameters) // 2
  A = X
  for l in range(1, L + 1):
    A_prev = A
    W = parameters[f'W{l}']
    b = parameters[f'b{l}']
    Z = np.dot(W, A_prev) + b
    Z=np.float128(Z)
    A = 1 / (1 + np.exp(-Z))
  AL = A
  return AL
#------------------------------
def accuracy (X, Y, parameters):
  AL = predict(X, parameters)
  #print(AL)
  predictions = (AL > 0.999).astype(int)
  accuracy = np.mean(predictions == Y)
  return accuracy
#------------------------------
def cost(Yp, Y):
    Yp = np.maximum(Yp, sys.float_info.min)
    m = Y.shape[1]
    cost = -1/m * np.sum(Y * np.log(Yp + sys.float_info.epsilon) + (1 - Y) * np.log(1 - Yp + sys.float_info.epsilon))
    cost = np.squeeze(cost)
    return cost

#------------------------------
def backward_step(dA, cache):
    A_1, A = cache
    dZ = dA * A * (1 - A)
    m = A_1.shape[1]

    dA_1 = np.dot(W.T, dZ)
    dW = (1/m) * np.dot(dZ, A_1.T)
    db = (1/m) * np.sum(dZ, axis=1, keepdims=True)

    return dA_1, dW, db
#------------------------------
def backward(AL, Y, caches):
    grads = {}
    L = len(caches)
    epsilon = 1e-12
    dAL = -(np.divide(Y, AL + epsilon) - np.divide(1 - Y, 1 - AL + epsilon))
    current_cache = caches[L - 1]
    A_prev, W, A = current_cache
    m = A_prev.shape[1]
    dZ = dAL * A * (1 - A)
    grads["dA" + str(L - 1)] = np.dot(W.T, dZ)
    grads["dW" + str(L)] = (1 / m) * np.dot(dZ, A_prev.T)
    grads["db" + str(L)] = (1 / m) * np.sum(dZ, axis=1, keepdims=True)

    for l in reversed(range(L - 1)):
        current_cache = caches[l]
        A_prev, W, A = current_cache

        dZ = grads["dA" + str(l + 1)] * A * (1 - A)
        grads["dA" + str(l)] = np.dot(W.T, dZ)
        grads["dW" + str(l + 1)] = (1 / m) * np.dot(dZ, A_prev.T)
        grads["db" + str(l + 1)] = (1 / m) * np.sum(dZ, axis=1, keepdims=True)

    return grads
#------------------------------
def update_parameters(parameters,grads, learning_rate):
    L= len(parameters)//2
    for l in range(L):
        parameters['W'+str(l+1)]= parameters['W'+str(l+1)]-learning_rate*grads['dW'+str(l+1)]
        parameters['b'+str(l+1)]= parameters['b'+str(l+1)]-learning_rate*grads['db'+str(l+1)]
    return parameters
#------------------------------
def train(X, Y, layers, learning_rate, iterations, print_cost):
    parameters = initialize_parameters(layers)
    for i in range(0, iterations):
        AL, caches = forward(X, parameters)
        current_cost = cost(AL, Y)
        grads = backward(AL, Y, caches)
        parameters = update_parameters(parameters, grads, learning_rate)

        if print_cost and i % print_cost == 0:
            print(f"Cost after iteration {i}: {current_cost}")
    print(f"Cost after iteration {iterations}: {current_cost}")
    return parameters

# **DATOS**

In [ ]:

import pandas as pd

archivo_csv = "/content/sample_data/spotify_songs.csv"
datos = pd.read_csv(archivo_csv)

#Filtrar el data frame sobre los parametros selectos para el entrenamiento de la red
dataFrame = datos[['danceability', 'energy','key','loudness','mode','speechiness','acousticness','instrumentalness','tempo','duration_ms']].to_numpy()
dataFrameGenre = datos['playlist_genre'].to_numpy()


#Se divide el data frame por genero

rockInput=dataFrame[11255-2:15216-1]
popInput=dataFrame[2-2:4408-1]
randbInput=dataFrame[21361-2:25706-1]




allGenresInput=rockInput
allGenresInput = np.append(allGenresInput,popInput,axis=0)
allGenresInput = np.append(allGenresInput,randbInput,axis=0)

allGenresInput = allGenresInput.T


genres = dataFrameGenre[11255-2:15216-1]
genres = np.append(genres,dataFrameGenre[2-2:4408-1],axis=0)
genres = np.append(genres,dataFrameGenre[21361-2:25706-1],axis=0)

Y=np.zeros((3,len(genres)))
Y[0]=np.where(genres[0:]== 'rock', 1, 0)
Y[1]=np.where(genres[0:]== 'pop', 1, 0)
Y[2]=np.where(genres[0:]== 'r&b', 1, 0)


print(np.shape(allGenresInput))
print(np.shape(Y))
print("Red 1 de [10,3] con 100 mil iteraciones:")
parameters1=train(allGenresInput, Y, [10,6,3], 0.1, 100000, 5000)

(10, 12715)
(3, 12715)
Red 1 de [10,3] con 100 mil iteraciones:
Cost after iteration 0: 2.0786749952115975
Cost after iteration 5000: 1.907913628628188
Cost after iteration 10000: 1.907913628601812
Cost after iteration 15000: 1.9079136285035516
Cost after iteration 20000: 1.907913276591425
Cost after iteration 25000: 1.907913056884645
Cost after iteration 30000: 1.907913628861846
Cost after iteration 35000: 1.9079136287489775
Cost after iteration 40000: 1.9079136287208107
Cost after iteration 45000: 1.9079136287080143
Cost after iteration 50000: 1.9079136287007041
Cost after iteration 55000: 1.9079136286959746
Cost after iteration 60000: 1.9079136286926643
Cost after iteration 65000: 1.9079136286902179
Cost after iteration 70000: 1.907913628688336
Cost after iteration 75000: 1.9079136286868434
Cost after iteration 80000: 1.907913628685631
Cost after iteration 85000: 1.9079136286846263
Cost after iteration 90000: 1.9079136286837803
Cost after iteration 95000: 1.907913628683058
Cost afte

In [ ]:
import pandas as pd

archivo_csv = "/content/sample_data/spotify_songs.csv"
datos = pd.read_csv(archivo_csv)

#Filtrar el data frame sobre los parametros selectos para el entrenamiento de la red
dataFrame = datos[['key','loudness','mode','speechiness','acousticness','instrumentalness','tempo']].to_numpy()
dataFrameGenre = datos['playlist_genre'].to_numpy()


#Se divide el data frame por genero


rockInput=dataFrame[11255-2:15216-1]
popInput=dataFrame[2-2:4408-1]
randbInput=dataFrame[21361-2:25706-1]




allGenresInput=rockInput
allGenresInput = np.append(allGenresInput,popInput,axis=0)
allGenresInput = np.append(allGenresInput,randbInput,axis=0)

allGenresInput = allGenresInput.T


genres = dataFrameGenre[11255-2:15216-1]
genres = np.append(genres,dataFrameGenre[2-2:4408-1],axis=0)
genres = np.append(genres,dataFrameGenre[21361-2:25706-1],axis=0)

Y=np.zeros((3,len(genres)))
Y[0]=np.where(genres[0:]== 'rock', 1, 0)
Y[1]=np.where(genres[0:]== 'pop', 1, 0)
Y[2]=np.where(genres[0:]== 'r&b', 1, 0)


print(np.shape(allGenresInput))
print(np.shape(Y))
print("Red 1 de [43] con 100 mil iteraciones:")
parameters1=train(allGenresInput, Y, [7,8,5,4,3], 0.1, 100000, 5000)

(7, 12715)
(3, 12715)
Red 1 de [43] con 100 mil iteraciones:
Cost after iteration 0: 2.0863526025962935
Cost after iteration 5000: 1.907913631540566
Cost after iteration 10000: 1.9079136298150996
Cost after iteration 15000: 1.9079136281055262
Cost after iteration 20000: 1.9079136264115428
Cost after iteration 25000: 1.9079136247323616
Cost after iteration 30000: 1.9079136230666522
Cost after iteration 35000: 1.9079136214125296
Cost after iteration 40000: 1.9079136197675781
Cost after iteration 45000: 1.907913618128905
Cost after iteration 50000: 1.9079136164932056
Cost after iteration 55000: 1.907913614856837
Cost after iteration 60000: 1.9079136132158836
Cost after iteration 65000: 1.9079136115662165
Cost after iteration 70000: 1.9079136099035383
Cost after iteration 75000: 1.9079136082234163
Cost after iteration 80000: 1.907913606521301
Cost after iteration 85000: 1.9079136047925354
Cost after iteration 90000: 1.9079136030323518
Cost after iteration 95000: 1.907913601235862
Cost afte

# **Test**

In [ ]:
#Se divide el data frame por genero

rockInput=dataFrame[15216-1:16205-1]
popInput=dataFrame[4408-1:5508-1]
randbInput=dataFrame[25706-1:26791-1]




allGenresInput=rockInput
allGenresInput = np.append(allGenresInput,popInput,axis=0)
allGenresInput = np.append(allGenresInput,randbInput,axis=0)

allGenresInput = allGenresInput.T

test=allGenresInput


genres = dataFrameGenre[15216-1:16205-1]
genres = np.append(genres,dataFrameGenre[4408-1:5508-1],axis=0)
genres = np.append(genres,dataFrameGenre[25706-1:26791-1],axis=0)

Y_true=np.zeros((3,len(genres)))
Y_true[0]=np.where(genres[0:]== 'rock', 1, 0)
Y_true[1]=np.where(genres[0:]== 'pop', 1, 0)
Y_true[2]=np.where(genres[0:]== 'r&b', 1, 0)

print("Red 1 de [10,6,3] con 100 mil iteraciones:")
print(accuracy(test,Y_true,parameters1))


Red 1 de [10,6,3] con 100 mil iteraciones:
0.6666666666666666
